# Bundle Sharing & Reproducibility with Amazon Braket

This notebook demonstrates devqubit's **portability and reproducibility** features.

**What you'll see:**
- Pack runs into self-contained bundles (ZIP files)
- Unpack bundles in a different environment
- Verify reproducibility across workspaces
- Cross-workspace run comparison

**The scenario:** You run an experiment locally, pack it, share with a collaborator, and they verify they can reproduce the results.

We use Braket's `LocalSimulator` (no AWS credentials needed).

In [ ]:
from importlib.metadata import entry_points

if not any(ep.name == "braket" for ep in entry_points(group="devqubit.adapters")):
    raise ImportError(
        "devqubit Braket adapter is not installed.\n"
        "Install with: pip install 'devqubit[braket]'"
    )
else:
    print("Braket adapter available!")

In [ ]:
from pathlib import Path
import shutil
import numpy as np

from braket.circuits import Circuit
from braket.devices import LocalSimulator

from devqubit import Config, set_config, track
from devqubit.compare import diff, verify_baseline
from devqubit.storage import create_store, create_registry
from devqubit.bundle import pack_run, unpack_bundle, Bundle
from devqubit.compare import VerifyPolicy

### Setup

We create **two separate workspaces** to simulate the collaboration scenario:

- **LOCAL**: Your machine where you run the original experiment
- **REMOTE**: Collaborator's machine where they import and verify

In practice, these would be on different computers. Here we simulate both locally.

In [ ]:
# Configuration
PROJECT = "qaoa_maxcut"
N_QUBITS = 4
P_LAYERS = 2
SHOTS = 2048
SEED = 42

# Two separate workspaces
LOCAL_ROOT = Path(".devqubit_local")
REMOTE_ROOT = Path(".devqubit_remote")

for p in [LOCAL_ROOT, REMOTE_ROOT]:
    if p.exists():
        shutil.rmtree(p)

# Setup LOCAL workspace (where we run the original experiment)
set_config(Config(root_dir=LOCAL_ROOT))
local_store = create_store()
local_registry = create_registry()

np.random.seed(SEED)
print(f"LOCAL workspace:  {LOCAL_ROOT.resolve()}")
print(f"REMOTE workspace: {REMOTE_ROOT.resolve()}")

### 1. QAOA Circuit for MaxCut

We use QAOA (Quantum Approximate Optimization Algorithm) for MaxCut on a ring graph.

**Problem:** Partition 4 nodes into two sets to maximize edges between sets.

For a ring graph with 4 nodes, the maximum cut is 4 (every edge crosses the partition).

**Circuit structure:**
1. Initial superposition (H on all qubits)
2. Cost layer: ZZ rotations on edges (parameterized by γ)
3. Mixer layer: RX rotations (parameterized by β)
4. Repeat for p layers

In [ ]:
def create_qaoa_circuit(n_qubits: int, p: int, gamma: float = 0.3, beta: float = 0.2):
    """Create QAOA circuit for MaxCut on a ring graph."""
    circuit = Circuit()

    # Initial superposition
    for q in range(n_qubits):
        circuit.h(q)

    # QAOA layers
    for layer in range(p):
        g = gamma + 0.1 * layer
        b = beta + 0.1 * layer

        # Cost: ZZ interactions on ring edges
        for q in range(n_qubits):
            circuit.zz(q, (q + 1) % n_qubits, g)

        # Mixer: X rotations
        for q in range(n_qubits):
            circuit.rx(q, 2 * b)

    return circuit


def maxcut_value(bitstring: str) -> int:
    """Compute cut value for ring graph."""
    n = len(bitstring)
    return sum(bitstring[i] != bitstring[(i + 1) % n] for i in range(n))


def compute_metrics(counts: dict, n_qubits: int) -> dict:
    """Compute QAOA metrics from measurement counts."""
    total = sum(counts.values())
    max_cut = n_qubits  # Ring graph max cut = n

    avg_cut = sum(maxcut_value(bs) * c for bs, c in counts.items()) / total
    best_bs = max(counts.keys(), key=lambda bs: (maxcut_value(bs), counts[bs]))

    return {
        "avg_cut": float(avg_cut),
        "approx_ratio": float(avg_cut / max_cut),
        "best_bitstring": best_bs,
        "best_cut": maxcut_value(best_bs),
    }

### 2. Run the Original Experiment (LOCAL)

We run QAOA on the local simulator and track everything:
- Problem parameters
- Circuit (via backend wrapping)
- Results and metrics

In [ ]:
circuit = create_qaoa_circuit(N_QUBITS, P_LAYERS)
device = LocalSimulator()

with track(project=PROJECT, run_name="reference") as run:
    backend = run.wrap(device)

    run.log_params(
        {
            "n_qubits": N_QUBITS,
            "p_layers": P_LAYERS,
            "shots": SHOTS,
            "problem": "maxcut_ring",
            "gamma": 0.3,
            "beta": 0.2,
        }
    )
    run.set_tags({"role": "reference", "algorithm": "qaoa"})

    # Execute circuit
    task = backend.run(circuit, shots=SHOTS)
    counts = dict(task.result().measurement_counts)

    # Compute and log metrics
    metrics = compute_metrics(counts, N_QUBITS)
    run.log_metrics(
        {
            "avg_cut": metrics["avg_cut"],
            "approx_ratio": metrics["approx_ratio"],
            "best_cut": metrics["best_cut"],
        }
    )
    run.log_json("counts", counts, role="result")

    reference_id = run.run_id

print(f"Reference run: {reference_id}")
print(f"Approximation ratio: {metrics['approx_ratio']:.3f}")
print(f"Best solution: {metrics['best_bitstring']} (cut = {metrics['best_cut']})")

### 3. Pack into Bundle

Now we **pack the run** into a portable bundle. The bundle is a ZIP file containing:

- Run metadata (params, metrics, tags)
- All artifacts (circuits, results)
- Object store data (by content hash)

Bundles are self-contained — they can be shared via email, Slack, or attached to PRs.

In [ ]:
bundle_path = LOCAL_ROOT / "qaoa_reference.devqubit.zip"
pack_result = pack_run(reference_id, bundle_path)

print(f"Bundle created: {bundle_path.name}")
print(f"  Location:  {bundle_path}")
print(f"  Size:      {bundle_path.stat().st_size / 1024:.1f} KB")
print(f"  Artifacts: {pack_result.artifact_count}")
print(f"  Objects:   {pack_result.object_count}")

### Inspect Bundle Contents

Before sending to a collaborator, you can inspect the bundle contents without fully extracting it.

In [ ]:
with Bundle(bundle_path) as b:
    print(f"Run ID:  {b.run_id}")
    print(f"Project: {b.get_project()}")
    print(f"Objects: {len(b.list_objects())}")

### 4. Unpack in REMOTE Workspace

Now we simulate a collaborator receiving the bundle. They:

1. Set up their own workspace
2. Unpack the bundle
3. The run is now available in their registry

In practice, the bundle would be transferred via file sharing, email, etc.

In [ ]:
# Setup REMOTE workspace (collaborator's environment)
remote_config = Config(root_dir=REMOTE_ROOT)
remote_store = create_store(config=remote_config)
remote_registry = create_registry(config=remote_config)

# Unpack the bundle
unpack_result = unpack_bundle(
    bundle_path,
    dest_store=remote_store,
    dest_registry=remote_registry,
)

print("Unpacked in REMOTE workspace:")
print(f"  Run ID:    {unpack_result.run_id}")
print(f"  Artifacts: {unpack_result.artifact_count}")
print(f"  Objects:   {unpack_result.object_count}")

### Verify Integrity

Let's verify that the unpacked run matches the original exactly.

In [ ]:
original = local_registry.load(reference_id)
imported = remote_registry.load(unpack_result.run_id)

print("Integrity Check:")
print(f"  Run IDs match: {original.run_id == imported.run_id}")
print(f"  Params match:  {original.params == imported.params}")
print(f"  Metrics match: {original.metrics == imported.metrics}")
print(f"  Tags match:    {original.tags == imported.tags}")

### 5. Reproduce and Verify (REMOTE)

The collaborator wants to verify they can reproduce the results. They:

1. Set the imported run as baseline
2. Run the same experiment with the same parameters
3. Verify the new run matches the baseline

In [ ]:
# Set imported run as baseline in REMOTE workspace
remote_registry.set_baseline(PROJECT, unpack_result.run_id)
print(f"Baseline set in REMOTE: {unpack_result.run_id}")

In [ ]:
# Switch to REMOTE config for tracking
set_config(remote_config)

with track(project=PROJECT, run_name="reproduction") as run:
    backend = run.wrap(LocalSimulator())

    # Use the same parameters as the imported run
    run.log_params(imported.params)
    run.set_tags({"role": "reproduction", "algorithm": "qaoa"})

    # Re-run the experiment
    circ = create_qaoa_circuit(N_QUBITS, P_LAYERS)
    task = backend.run(circ, shots=SHOTS)
    counts = dict(task.result().measurement_counts)

    metrics = compute_metrics(counts, N_QUBITS)
    run.log_metrics(
        {
            "avg_cut": metrics["avg_cut"],
            "approx_ratio": metrics["approx_ratio"],
            "best_cut": metrics["best_cut"],
        }
    )
    run.log_json("counts", counts, role="result")

    repro_id = run.run_id

print(f"Reproduction run: {repro_id}")
print(f"Approximation ratio: {metrics['approx_ratio']:.3f}")

In [ ]:
# Verify reproduction against baseline
policy = VerifyPolicy(
    params_must_match=True,  # Same configuration required
    program_must_match=True,  # Same circuit structure required
    tvd_max=0.10,  # Allow 10% distribution difference (shot noise)
)

result = verify_baseline(
    repro_id,
    project=PROJECT,
    policy=policy,
    store=remote_store,
    registry=remote_registry,
)

print(f"Verification: {'[OK] PASSED' if result.ok else '[X] FAILED'}")
if not result.ok:
    print("Failures:")
    for f in result.failures:
        print(f"  - {f}")

---
## 6. Cross-Workspace Comparison

The `diff()` function can compare runs even when they're in different workspaces (you just need to specify the stores).

In [ ]:
comparison = diff(
    unpack_result.run_id,
    repro_id,
    store=remote_store,
    registry=remote_registry,
)

print("Imported Reference vs. Reproduction")
print("=" * 50)
print(comparison)

---
## 7. Query REMOTE Registry

Let's see what runs are now in the REMOTE workspace.

In [ ]:
print("REMOTE Workspace Runs")
print("=" * 60)

for info in remote_registry.list_runs(project=PROJECT):
    rec = remote_registry.load(info["run_id"])
    role = rec.tags.get("role", "-")
    approx = rec.metrics.get("approx_ratio", 0)

    print(f"  {rec.run_id[:12]}  {role:12s}  approx_ratio={approx:.3f}")

baseline = remote_registry.get_baseline(PROJECT)
print(f"\nBaseline: {baseline['run_id'][:16]}...")

---
## Summary

In this notebook we demonstrated the complete bundle workflow:

| Step | What we did | Location |
|------|-------------|----------|
| 1. Run experiment | QAOA on LocalSimulator | LOCAL |
| 2. Pack bundle | `pack_run()` → ZIP file | LOCAL |
| 3. Inspect bundle | `Bundle()` context manager | (portable) |
| 4. Unpack bundle | `unpack_bundle()` | REMOTE |
| 5. Set baseline | `registry.set_baseline()` | REMOTE |
| 6. Reproduce | Run same experiment | REMOTE |
| 7. Verify | `verify_baseline()` | REMOTE |

**Key insight:** Bundles enable reproducible science. Anyone can unpack a bundle, reproduce the experiment, and verify the results match.

In [ ]:
shutil.rmtree(LOCAL_ROOT, ignore_errors=True)
shutil.rmtree(REMOTE_ROOT, ignore_errors=True)
print("Done!")